# Loading and Saving State 

So far, we have discussed how to build components in a multi-agent application - agents, teams, termination conditions. In many cases, it is useful to save the state of these components to disk and load them back later. This is particularly useful in a web application where stateless endpoints respond to requests and need to load the state of the application from persistent storage.

In this notebook, we will discuss how to save and load the state of agents, teams, and termination conditions. 
 

# Saving and Loading Agents

We can get the state of an agent by calling `save_state` method on the agent.

In [1]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.task import Console, MaxMessageTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_ext.models import OpenAIChatCompletionClient

assistant_agent = AssistantAgent(
    name="assistant_agent",
    system_message="You are a helpful assistant",
    model_client=OpenAIChatCompletionClient(
        model="gpt-4o-2024-08-06",
    ),
)

result = await assistant_agent.run(task="Write a 3 line poem on lake tangayika")
print(result.messages[-1].content)

In Tanganyika's depths, whispers of yore,  
Cradle of life where waters softly roar,  
Infinite horizon, nature's grand encore.  


In [4]:
agent_state = await assistant_agent.save_state()
print(agent_state)
# convert to dict/json 
print(vars(agent_state))

AssistantAgentState(state_type='AssistantAgent', version='1.0.0', model_context=[UserMessage(content='Write a 3 line poem on lake tangayika', source='user'), AssistantMessage(content="In Tanganyika's depths, whispers of yore,  \nCradle of life where waters softly roar,  \nInfinite horizon, nature's grand encore.  ", source='assistant_agent')])
{'state_type': 'AssistantAgent', 'version': '1.0.0', 'model_context': [UserMessage(content='Write a 3 line poem on lake tangayika', source='user'), AssistantMessage(content="In Tanganyika's depths, whispers of yore,  \nCradle of life where waters softly roar,  \nInfinite horizon, nature's grand encore.  ", source='assistant_agent')]}


In [5]:
new_assistant_agent = AssistantAgent(
    name="assistant_agent",
    system_message="You are a helpful assistant",
    model_client=OpenAIChatCompletionClient(
        model="gpt-4o-2024-08-06",
    ),
)
await new_assistant_agent.load_state(agent_state)
result = await new_assistant_agent.run(task="What was the last line of the previous poem you wrote")
print(result.messages[-1].content)

The last line of the poem is:  
"Infinite horizon, nature's grand encore."


Note: Agent state mainly consists of the model_context as the agent's state. If your agent has additional state you would like to save and load, consider overriding the `save_state` and `load_state` methods.
 

## Saving and Loading Teams 

We can get the state of a team by calling `save_state` method on the team and load it back by calling `load_state` method on the team. 

A team typically is composed of agents and some termination conditions. When we call `save_state` on a team, it saves the state of all the agents in the team and the termination conditions.  

We will begin by creating a simple RoundRobinGroupChat team with a single agent and ask it to write a poem. 


In [6]:
assistant_agent = AssistantAgent(
    name="assistant_agent",
    system_message="You are a helpful assistant",
    model_client=OpenAIChatCompletionClient(
        model="gpt-4o-2024-08-06",
    ),
)
# Define a team
agent_team = RoundRobinGroupChat([assistant_agent], termination_condition=MaxMessageTermination(max_messages=2))

# Run the team and stream messages to the console
stream = agent_team.run_stream(task="Write a beautiful poem 3-line about lake tangayika")
await Console(stream)
team_state = await agent_team.save_state()

---------- user ----------
Write a beautiful poem 3-line about lake tangayika
---------- assistant_agent ----------
Whispers of the deep, Tanganyika's embrace,  
Mirror of stars and time, in tranquil grace,  
Boundless tales within its azure face.  
[Prompt tokens: 29, Completion tokens: 32]
---------- Summary ----------
Number of messages: 2
Finish reason: Maximum number of messages 2 reached, current message count: 2
Total prompt tokens: 29
Total completion tokens: 32
Duration: 0.76 seconds


If we reset the team (simulating instantiation of the team),  and ask the question `What was the last line of the poem you wrote?`, we see that the team is unable to accomplish this as there is no reference to the previous run.

In [7]:
await agent_team.reset()
stream = agent_team.run_stream(task="What was the last line of the poem you wrote?")
await Console(stream)

---------- user ----------
What was the last line of the poem you wrote?
---------- assistant_agent ----------
I'm sorry, but I don't have the ability to recall specific lines from poems I've previously written or discussed since I don't have memory. However, I'd be happy to help you create a new poem or discuss any poetry-related topics you're interested in!
[Prompt tokens: 28, Completion tokens: 48]
---------- Summary ----------
Number of messages: 2
Finish reason: Maximum number of messages 2 reached, current message count: 2
Total prompt tokens: 28
Total completion tokens: 48
Duration: 0.84 seconds


TaskResult(messages=[TextMessage(source='user', models_usage=None, content='What was the last line of the poem you wrote?'), TextMessage(source='assistant_agent', models_usage=RequestUsage(prompt_tokens=28, completion_tokens=48), content="I'm sorry, but I don't have the ability to recall specific lines from poems I've previously written or discussed since I don't have memory. However, I'd be happy to help you create a new poem or discuss any poetry-related topics you're interested in!")], stop_reason='Maximum number of messages 2 reached, current message count: 2')

Next, we load the state of the team and ask the same question. We see that the team is able to accurately return the last line of the poem it wrote.

Note: You can serialize the state of the team to a file and load it back later.

In [ ]:
from autogen_agentchat.state import BaseTeamState

# convert team state to dict (save to file if needed)
team_state_dict = vars(team_state)
print(team_state_dict)      

# convert back to team state object
team_state = BaseTeamState(**vars(team_state)) 

# load team state
await agent_team.load_state(team_state)
stream = agent_team.run_stream(task="What was the last line of the poem you wrote?")
await Console(stream)

{'state_type': 'BaseTeamState', 'version': '1.0.0', 'agent_names': ['assistant_agent'], 'termination_state': MaxMessageTerminationState(state_type='MaxMessageTerminationState', version='1.0.0', terminated=False, message_count=0, max_messages=2), 'agent_states': {'assistant_agent': AssistantAgentState(state_type='AssistantAgent', version='1.0.0', model_context=[UserMessage(content='Write a beautiful poem 3-line about lake tangayika', source='user'), AssistantMessage(content="Whispers of the deep, Tanganyika's embrace,  \nMirror of stars and time, in tranquil grace,  \nBoundless tales within its azure face.  ", source='assistant_agent')])}, 'manager_state': BaseGroupChatManagerState(state_type='BaseGroupChatManagerState', version='1.0.0', message_thread=[], current_turn=0)}
---------- user ----------
What was the last line of the poem you wrote?
---------- assistant_agent ----------
The last line of the poem I wrote is: "Boundless tales within its azure face."
[Prompt tokens: 84, Complet

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='What was the last line of the poem you wrote?'), TextMessage(source='assistant_agent', models_usage=RequestUsage(prompt_tokens=84, completion_tokens=19), content='The last line of the poem I wrote is: "Boundless tales within its azure face."')], stop_reason='Maximum number of messages 2 reached, current message count: 2')

{'state_type': 'BaseTeamState', 'version': '1.0.0', 'agent_names': ['assistant_agent'], 'termination_state': MaxMessageTerminationState(state_type='MaxMessageTerminationState', version='1.0.0', terminated=False, message_count=0, max_messages=2), 'agent_states': {'assistant_agent': AssistantAgentState(state_type='AssistantAgent', version='1.0.0', model_context=[UserMessage(content='Write a beautiful poem 3-line about lake tangayika', source='user'), AssistantMessage(content="Whispers of the deep, Tanganyika's embrace,  \nMirror of stars and time, in tranquil grace,  \nBoundless tales within its azure face.  ", source='assistant_agent')])}, 'manager_state': BaseGroupChatManagerState(state_type='BaseGroupChatManagerState', version='1.0.0', message_thread=[], current_turn=0)}
BaseTeamState(state_type='BaseTeamState', version='1.0.0', agent_names=['assistant_agent'], termination_state=MaxMessageTerminationState(state_type='MaxMessageTerminationState', version='1.0.0', terminated=False, mess

## Saving and Loading Termination Conditions

We can get the state of a termination condition by calling `save_state` method on the termination condition and load it back by calling `load_state` method on the termination condition. 

In [15]:
max_termination = MaxMessageTermination(max_messages=2)
text_termination = TextMentionTermination(text="stop")
termination = text_termination | max_termination
termination_state = await termination.save_state()
print(termination_state)
await termination.load_state(termination_state)

OrTerminationState(state_type='OrTerminationState', version='1.0.0', terminated=False, condition_states=[TextMentionTerminationState(state_type='TextMentionTerminationState', version='1.0.0', terminated=False, text='stop'), MaxMessageTerminationState(state_type='MaxMessageTerminationState', version='1.0.0', terminated=False, message_count=0, max_messages=2)])
